# Проверка Runnable-объектов

В этом разделе приведены примеры того, как можно получить более подробную информацию о работе Runnable.

Для демонстрации используется Runnable, который реализует RAG-подход — извлекает данные и выполняет поиск по ним.

In [ ]:
%pip install --upgrade --quiet  gigachain faiss-cpu tiktoken

In [1]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.chat_models.gigachat import GigaChat
from langchain_community.embeddings.gigachat import GigaChatEmbeddings

In [2]:
vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=GigaChatEmbeddings(credentials="<авторизационные_данные>", verify_ssl_certs=False)
)
retriever = vectorstore.as_retriever()

template = """Отвечай на вопросы с учетом контекста:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = GigaChat(credentials="<авторизационные_данные>", verify_ssl_certs=False)

In [3]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

## Доступ к графу работы

Для получения графа работы Runnable, используйте метод `get_graph()`.

In [ ]:
chain.get_graph()

## Печать графа работы

Хоть это может быть и не очень удобно, для большей наглядности граф можно распечатать в ascii-кодировке с помощью метода `print_ascii()`.

In [5]:
chain.get_graph().print_ascii()

           +---------------------------------+         
           | Parallel<context,question>Input |         
           +---------------------------------+         
                    **               **                
                 ***                   ***             
               **                         **           
+----------------------+              +-------------+  
| VectorStoreRetriever |              | Passthrough |  
+----------------------+              +-------------+  
                    **               **                
                      ***         ***                  
                         **     **                     
           +----------------------------------+        
           | Parallel<context,question>Output |        
           +----------------------------------+        
                             *                         
                             *                         
                             *                  

## Доступ к промптам

Для получения промптов, которые используются внутри цепочки, вызовите метод `get_prompts()`.

In [6]:
chain.get_prompts()

[ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'))])]